In [1]:
from pathlib import Path
import os

from tqdm import tqdm, tqdm_notebook

import numpy as np
import pandas as pd

%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns

from textblob import TextBlob
from textblob.translate import NotTranslated
from joblib import Parallel, delayed

from googletrans import Translator

In [18]:
DATA_PATH = Path('../data')

plt.rcParams['figure.figsize'] = (10, 8)

tqdm_notebook().pandas()

In [3]:
NAN_WORD = "xxempty"

In [4]:
train_df = pd.read_csv(DATA_PATH/'train.csv').fillna(NAN_WORD)
valid_df = pd.read_csv(DATA_PATH/'valid.csv').fillna(NAN_WORD)
test_df = pd.read_csv(DATA_PATH/'test.csv', index_col=0).fillna(NAN_WORD)

In [21]:
def translate(comment, translator, language):
    if hasattr(comment, 'decode'):
        comment = comment.decode('utf-8')

    try:
        text = translator.translate(comment, dest=language).text
        text = translator.translate(text, dest='en').text
    except NotTranslated:
        pass

    return str(text)

In [ ]:
translator = Translator(service_urls=[
#       'translate.google.com',
      'translate.google.co.kr',
    ])

train_df['title'].head().progress_apply(translate, translator=translator, language='es')

In [30]:
def backtranslate(file_path, file_type, languages, thread_count=1, result_path=DATA_PATH):
    df = pd.read_csv(file_path)
    titles_list = df['title'].fillna(NAN_WORD).values
    texts_list = df['text'].fillna(NAN_WORD).values

    if not os.path.exists(result_path):
        os.mkdir(result_path)

    parallel = Parallel(thread_count, backend='threading', verbose=5)
    
    translator = Translator()
                        
    for language in tqdm_notebook(languages):
        print(f'Translate comments using "{language}" language')
        translated_titles = parallel(delayed(translate)(title, translator, language) for title in titles_list)
        translated_texts = parallel(delayed(translate)(text, translator, language) for text in texts_list)
        
        df["title"] = translated_title
        df["text"] = translated_text

        train_data.to_csv(result_path/r'{file_type}_{language}.csv', index=False)

In [26]:
languages = ['es', 'de', 'fr']

In [27]:
backtranslate(DATA_PATH/'valid.csv', 'valid', languages)

Translate comments using "es" language


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


JSONDecodeError: Expecting value: line 1 column 1 (char 0)